In [ ]:
# Download dependencies
!pip3 install pandas
!pip3 install matplotlib
!pip3 install adjustText

# Generate graph from the actual results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib import rcParams

rcParams.update({'font.size': 18})
rcParams['font.family'] = 'Times New Roman'
texts = []
data = pd.read_csv("../results-march-2023/metrics/graphData.csv")
text = ["cdx","depscan","jbom","cdx-mp", "or","big"]
data.shape
data.head()
g1 = data
D_P = g1['D_P']
D_R = g1['D_R']
T_P = g1['T_P']
T_R = g1['T_R']
fig = plt.figure()
fig, ax1 = plt.subplots(1, sharex=True, sharey=True)
ax1.scatter(D_P,D_R, c='b', marker='x')
ax1.scatter(T_P,T_R, c='r')
plt.xlabel('Precision in [%]', fontsize=14)
plt.ylabel('Recall in [%]', fontsize=14)
plt.ylim([0, 100])
plt.xlim([0, 100])
# Loop for annotation of all points
for i in range(len(D_P)):
    texts.append(plt.text(D_P[i], D_R[i], text[i]))
for i in range(len(D_P)):
    texts.append(plt.text(T_P[i], T_R[i], text[i]))



adjust_text(texts,# x=D_P+T_P, y=D_R+T_R, autoalign='x',
            #force_points=5,
            #force_text=True,
            )
plt.savefig("resultGraphic.pdf", format="pdf", bbox_inches="tight")
plt.show()